# 형태소 분석 기반 토큰화의 문제

-   형태소 분석기는 작성된 알고리즘 또는 학습된 내용을 바탕으로 토큰화를 하기 때문에 오탈자나 띄어쓰기 실수, 신조어, 외래어, 고유어 등이 사용된 경우 제대로 토큰화 하지 못한다.
-   그래서 발생 할 수있는 잠재적 문제점
    -   어휘사전을 크게 만든다.
        -   같은 의미의 단어가 형태소 분석이 안되어 여러개 등록될 수있다.
        -   ex) 신조어 `돈쭐` 이라는 단어를 인식 못할 경우 `"돈쭐내러", "돈쭐나", "돈쭐냄"` 등이 다 등록 될 수 있다.
    -   OOV(Out Of Vocab)에 대응하기 어렵게 만든다. - 같은 어근의 단어가 있지만 조사등이 바뀐 신조어등을 OOV로 인식할 수있다.
        > -   Out Of Vocab(OOV)
        >     -   Vocab(어휘사전): 코퍼스를 구성하는 토큰들의 모음이다.
        >     -   OOV는 어휘사전에 없는 토큰을 말한다.

# Subword Tokenization(하위 단어 토큰화)

## 개념

-   하나의 단어를 작은 단위의 하위단어(subword)들의 조합으로 나누는 방식으로 토큰화 한다.
    -   자주 쓰이는 문자조합은 더 작은 subword로 분리하면 안된다.
    -   희귀한 문자조합은 의미있는 더 작은 subword로 분리되어야 한다
    -   ex) normally 가 희귀 단어로 분류되면 'normal'과 'ly' 로 분리한다.

## 잇점

-   token 단어의 길이를 줄일 수 있어 처리 속도가 빨라진다.
-   OOV 문제, 신조어, 은어, 고유어등의 문제를 완화할 수 있다.

## 방법

-   Byte Pair Encoding (BPE), Word Piece, Unigram Model 등


# Byte Pair Encoding

-   원래 Text data 압축을 위해 만들어진 방법으로 text 에서 많이 등장하는 두글자 쌍의 조합을 찾아 부호화하는 알고리즘이다.
-   연속된 글자 쌍이 더 나타나지 않거나 정해진 어휘사전 크기에 도달 할 때 까지 조합을 찾아 부호화 하는 작업을 반복한다.

## text 압축 방식의 예

-   원문: abracadabra

1. AracadAra: ab -> A :=> 원문에서 가장 빈도수 많은 ab를 A(부호로 아무 글자나 사용할 수 있다.)로 치환
2. ABcadAB: ra -> B :=> 1에서 가장 빈도수가 많은 ra를 B로 치환
3. CcadC: AB -> C :=> 2에서 가장 빈도수 많은 AB를 C로 치환한다.(치환된 글자 쌍도 변환대상에 포함된다.)

## BPE Tokenizer 방식

BPE 토크나이저는 자주 등장하는 글자 쌍을 찾아 치환하는 대신 **단어 사전**에 추가한다.

### 예)

1. 말뭉치의 토큰들의 빈도수, 어휘사전은 아래와 같을 경우
    - 빈도사전: ('low', 5), ('lower', 2), ('newest', 6), ('widest', 3)
    - 어휘사전: ['low', 'lower', 'newest', 'widest']
2. 빈도 사전내의 모든 단어들을 글자 단위로 나눈다.
    - 빈도사전: ('l', 'o', 'w', 5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w']
3. 빈도 사전을 기준으로 가장 자주 등장하는 글자 쌍(byte pair)를 찾는다. 위에서는 **'e'와 's'가 총 9번으로 가장 많이 등장함**. 'e'와 's'를 'es'로 합치고 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w', 5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'es'**, 't', 6), ('w', 'i', 'd', **'es'**, 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**]
4. 3 번의 과정을 계속 반복한다. 빈도수가 가장 많은 'es'와 't' 쌍을 'est'로 병합하고 'est'를 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w', 5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**]
5. 만약 10번 반복했다고 하면 다음과 같은 빈도 사전과 어휘 사전이 생성된다.
    - 빈도 사전: (**'low'**, 5), (**'low'**, 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**, **'lo'**,**'low'**, **'low'**, **'ne'**, **'new'**, **'newest'**, **'wi'**, **'wid'**, **'widest'**]

-   위와 같이 어휘 사전이 만들어 지면 원래 어휘사전에 없던 것들에 대한 처리를 할 수있다.
    -   ex)
        -   'newer' :=> 'new', 'e', 'r',
        -   'lowest' :=> 'low', 'est'
        -   'wider' :=> 'wid', 'e', 'r'


# 데이터 조회

-   Korpora Dataset
    -   2017년 8월 부터 2019년 3월 까지 청와대 청원 게시판에 올라온 텍스트 말뭉치
    -   설치: `pip install korpora`


In [2]:
# !pip install korpora

In [3]:
from Korpora import Korpora
corpus = Korpora.load("korean_petitions")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/petitions_archive
    References :

    청와대 국민청원 게시판의 데이터를 월별로 수집한 것입니다.
    청원은 게시판에 글을 올린 뒤, 한달 간 청원이 진행됩니다.
    수집되는 데이터는 청원종료가 된 이후의 데이터이며, 청원 내 댓글은 수집되지 않습니다.
    단 청원의 동의 개수는 수집됩니다.
    자세한 내용은 위의 repository를 참고하세요.

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[korean_petitions] download petitions_2017-08: 1.84MB [00:00, 41.4MB/s]
[korean_petitions] download petitions_2017-09: 20.4MB [00:00, 84.3MB/s]                                
[korean_petitions] download petitions_2017-10: 12.0MB [00:00, 54.4MB/s]                                
[korean_petitions] download petitions_2017-11: 28.4MB [00:01, 28.0MB/s]                                
[korean_petitions] download petitions_2017-12: 29.0MB [00:00, 34.2MB/s]                                
[korean_petitions] download petitions_2018-01: 43.9MB [00:02, 18.4MB/s]                                
[korean_petitions] download petitions_2018-02: 33.8MB [00:00, 59.7MB/s]                                
[korean_petitions] download petitions_2018-03: 34.3MB [00:01, 23.7MB/s]                                
[korean_petitions] download petitions_2018-04: 35.5MB [00:01, 22.1MB/s]                                
[korean_petitions] download petitions_2018-05: 37.5MB [00:01, 21.2MB/s]                         

In [4]:
type(corpus)

Korpora.korpus_korean_petitions.KoreanPetitionsKorpus

In [6]:
petitions = corpus.get_all_texts()
print(type(petitions), len(petitions))

<class 'list'> 433631


In [12]:
petitions[3000]

'청소년보호법을 악용하는 사람들이 늘고있습니다 반드시 폐지해서 더 이상 이런끔찍한일을 보지않게 해주세요..'

In [13]:
# 파일에 하나의 말뭉치로 저장.
import os
os.makedirs("data", exist_ok=True)
path = "data/petitions_corpus.txt"
with open(path, "wt", encoding='utf-8') as fw:
    for txt in petitions:
        fw.write(txt+"\n")

# Sentencepiece

-   구글에서 개발한 subword tokenizer 라이브러리로 BPE 인코딩 방식과 유사한 알고리즘을 이용해 토큰화하고 단어 사전을 생성한다.
-   설치: `pip install sentencepiece`


In [14]:
!pip install sentencepiece

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   - -------------------------------------- 30.7/991.5 kB 1.3 MB/s eta 0:00:01
   ----------- ---------------------------- 276.5/991.5 kB 3.4 MB/s eta 0:00:01
   --------------------------------------- 991.5/991.5 kB 10.4 MB/s eta 0:00:00


In [15]:
# 단어사전 등 학습 결과 저장할 디렉토로
os.makedirs("saved_models")

## Train

-   가지고 있는 말뭉치를 이용해 tokenzier를 만든다.
-   `SentencePieceTrainer.Train()`
-   Parameter
    -   input: 학습데이터 경로
    -   model_prefix: 학습된 모델 파일 저장이름
    -   vocab_size: 어휘 사전 크기
    -   model_type: 토크나이저 알고리즘 선택 - 'unigram', 'bpe', 'char', 'word'
    -   max_sentence_length: 최대 문장 길이

### 학습 결과파일

-   model_prefix지정파일명.model : 학습된 tokenizer
-   model_prefix지정파일명.vocab : 어휘사전이 저장된 파일 (text 파일)

> token에 `_` 가 붙은 것(under score가 아님. LOWER ONE EIGHTH BLOCK 문자로 unicode \u2581 이다.)
> sentencepiece는 공백을 토큰화 하는 과정에 `_` 로 표현한다. 그래서 'hello world'는 'hello_world'로 처리해 토큰화 한다.


In [20]:
print("_가")
print("\u2581가")

_가
▁가


In [16]:
from sentencepiece import SentencePieceTrainer      # 어휘사전을 만드는 클래스
from sentencepiece import SentencePieceProcessor  # Tokenizer

In [18]:
import time
s = time.time()
SentencePieceTrainer.Train(
    "--input=data/petitions_corpus.txt \
    --model_prefix=saved_models/petitions_bpe \
    --vocab_size=10000 \
    --model_type=bpe \
    --unk_id=0"
)
e = time.time()
print(f'걸린시간: {e-s}초')

걸린시간: 108.50059247016907초


## Tokenizer

-   `SentencePieceProcessor`
    -   SentencePieceTrainer.Train()으로 학습한 모델을 이용해 Tokenizer생성


In [21]:
tokenizer = SentencePieceProcessor()
# 저장된 학습 모델 파일을 로딩
tokenizer.load("saved_models/petitions_bpe.model")

True

In [23]:
txt = petitions[0]
print(tokenizer.EncodeAsPieces(txt))

['▁안녕하세요', '.', '▁현재', '▁사', '대', ',', '▁교', '대', '▁등', '▁교', '원', '양', '성', '학교', '들의', '▁예비', '교사', '들이', '▁임용', '절', '벽', '에', '▁매우', '▁힘들어', '▁하고', '▁있는', '▁줄', '로', '▁압니다', '.', '▁정부', '▁부처', '에서는', '▁영양', '사의', '▁영양', "'", '교사', "'", '화', ',', '▁폭', '발', '적인', '▁영양', "'", '교사', "'", '▁채용', ',', '▁기간제', '▁교사', ',', '▁영', '전', '강', ',', '▁스', '강', '의', '▁무기계약', '직', '화가', '▁그들의', '▁임용', '▁절', '벽', '과는', '▁전혀', '▁무관', '한', '▁일이라고', '▁주장', '하고', '▁있지만', '▁조금만', '▁생각해', '보면', '▁전혀', '▁설득', '력', '▁없는', '▁말', '이라고', '▁생각합니다', '.', '▁학교', '▁수가', '▁같고', ',', '▁학생', '▁수가', '▁동일', '한데', '▁영양', '교사', '와', '▁기간제', '▁교사', ',', '▁영', '전', '강', '▁스', '강', '이', '▁학교에', '▁늘어나', '게', '▁되면', '▁당연히', '▁정', '규', '▁교', '원의', '▁수는', '▁줄어들', '게', '▁되지', '▁않겠습니까', '?', '▁기간제', '▁교사', ',', '▁영', '전', '강', ',', '▁스', '강', '의', '▁무기계약', '직', '화', ',', '▁정규직', '화', '▁꼭', '▁전면', '▁백', '지', '화', '해주십시오', '.', '▁백', '년대', '계', '인', '▁국가의', '▁교육', '에', '▁달', '린', '▁문제입니다', '.', '▁단순히', '▁대통령님의', '▁일자리', '▁공약', ',

In [24]:
txt2 = "이정후는 22일(한국시간) 미국 캘리포니아주 샌프란시스코의 오라클 파크에서 열린 애리조나 다이아몬드백스와 2024 메이저리그(MLB) 홈경기에 1번 타자 겸 중견수로 선발 출전, 4타석 2타수 무안타 2사사구를 기록했다. 시즌 타율은 0.289에서 0.282로 떨어졌다."
print(tokenizer.EncodeAsPieces(txt2))

['▁이', '정', '후', '는', '▁22', '일', '(', '한국', '시간', ')', '▁미국', '▁', '캘', '리', '포', '니', '아주', '▁', '샌', '프', '란', '시', '스', '코', '의', '▁오', '라', '클', '▁파', '크', '에서', '▁열', '린', '▁애', '리', '조', '나', '▁다', '이', '아', '몬', '드', '백', '스', '와', '▁20', '24', '▁메', '이', '저', '리', '그', '(', 'M', 'L', 'B', ')', '▁홈', '경', '기에', '▁1', '번', '▁타', '자', '▁겸', '▁중', '견', '수로', '▁선발', '▁출전', ',', '▁4', '타', '석', '▁2', '타', '수', '▁무', '안', '타', '▁2', '사', '사', '구를', '▁기록', '했다', '.', '▁시', '즌', '▁타', '율', '은', '▁0', '.2', '8', '9', '에서', '▁0', '.2', '8', '2', '로', '▁떨어', '졌다', '.']


In [25]:
print(tokenizer.EncodeAsIds(txt))
# 각 토큰에 부여된 ID(정수) 로 토큰화.

[667, 8515, 248, 9, 8533, 8553, 121, 8533, 138, 121, 8557, 8724, 8592, 413, 104, 2416, 2001, 39, 6015, 8774, 9148, 8522, 1187, 1576, 168, 55, 310, 8527, 3271, 8515, 132, 4457, 355, 7973, 967, 7973, 8763, 2001, 8763, 8630, 8553, 352, 8651, 164, 7973, 8763, 2001, 8763, 1320, 8553, 7318, 1337, 8553, 234, 8572, 8682, 8553, 356, 8682, 8526, 7584, 8643, 1460, 2180, 6015, 409, 9148, 2989, 1034, 5341, 8525, 8048, 1429, 22, 1142, 4933, 1580, 1219, 1034, 7490, 8677, 228, 68, 331, 196, 8515, 389, 1625, 8445, 8553, 328, 1625, 1961, 2147, 7973, 2001, 8628, 7318, 1337, 8553, 234, 8572, 8682, 356, 8682, 8513, 3734, 2242, 8552, 1378, 1500, 15, 8802, 121, 1310, 3751, 3451, 8552, 932, 4866, 8581, 7318, 1337, 8553, 234, 8572, 8682, 8553, 356, 8682, 8526, 7584, 8643, 8630, 8553, 1764, 8630, 533, 3870, 1296, 8520, 8630, 3288, 8515, 1296, 4939, 8646, 8538, 1781, 335, 8522, 497, 8809, 3728, 8515, 4084, 4119, 740, 2556, 8553, 570, 8582, 8555, 8629, 740, 2528, 2556, 8521, 1272, 415, 8540, 338, 8513, 9167, 19, 

In [28]:
e1 = tokenizer.EncodeAsPieces(txt2)
print(e1)
# 인코딩(토큰화)된 것을 원래 문장으로 복원.
d1 = tokenizer.DecodePieces(e1)
print(d1)

['▁이', '정', '후', '는', '▁22', '일', '(', '한국', '시간', ')', '▁미국', '▁', '캘', '리', '포', '니', '아주', '▁', '샌', '프', '란', '시', '스', '코', '의', '▁오', '라', '클', '▁파', '크', '에서', '▁열', '린', '▁애', '리', '조', '나', '▁다', '이', '아', '몬', '드', '백', '스', '와', '▁20', '24', '▁메', '이', '저', '리', '그', '(', 'M', 'L', 'B', ')', '▁홈', '경', '기에', '▁1', '번', '▁타', '자', '▁겸', '▁중', '견', '수로', '▁선발', '▁출전', ',', '▁4', '타', '석', '▁2', '타', '수', '▁무', '안', '타', '▁2', '사', '사', '구를', '▁기록', '했다', '.', '▁시', '즌', '▁타', '율', '은', '▁0', '.2', '8', '9', '에서', '▁0', '.2', '8', '2', '로', '▁떨어', '졌다', '.']
이정후는 22일(한국시간) 미국 캘리포니아주 샌프란시스코의 오라클 파크에서 열린 애리조나 다이아몬드백스와 2024 메이저리그(MLB) 홈경기에 1번 타자 겸 중견수로 선발 출전, 4타석 2타수 무안타 2사사구를 기록했다. 시즌 타율은 0.289에서 0.282로 떨어졌다.


In [30]:
e2 = tokenizer.EncodeAsIds(txt2)
print(e2)
# 복원
d2 = tokenizer.DecodeIds(e2)
print(d2)

[4, 8541, 8722, 8516, 4675, 8562, 8702, 1488, 266, 8696, 674, 8512, 0, 8550, 8784, 8517, 8307, 8512, 9821, 8881, 8833, 8535, 8638, 8964, 8526, 118, 8545, 9300, 425, 8880, 18, 392, 8809, 505, 8550, 8598, 8544, 38, 8513, 8537, 9603, 8665, 8938, 8638, 8628, 211, 5743, 1355, 8513, 8590, 8550, 8559, 8702, 9108, 9184, 9018, 8696, 3039, 8594, 405, 43, 8705, 552, 8540, 7665, 84, 8897, 2709, 4261, 7463, 8553, 216, 8815, 8918, 56, 8815, 8549, 61, 8576, 8815, 56, 8529, 8529, 2155, 3077, 1008, 8515, 40, 9554, 552, 8949, 8532, 1614, 2192, 8753, 8790, 18, 1614, 2192, 8753, 8602, 8527, 1120, 5101, 8515]
이정후는 22일(한국시간) 미국  ⁇ 리포니아주 샌프란시스코의 오라클 파크에서 열린 애리조나 다이아몬드백스와 2024 메이저리그(MLB) 홈경기에 1번 타자 겸 중견수로 선발 출전, 4타석 2타수 무안타 2사사구를 기록했다. 시즌 타율은 0.289에서 0.282로 떨어졌다.


In [33]:
### 학습된 모델 정보
print("총토큰개수:", tokenizer.GetPieceSize())
print("id->piece:", tokenizer.IdToPiece(4))
print("piece -> id:", tokenizer.PieceToId('경기'))

총토큰개수: 10000
id->piece: ▁이
piece -> id: 5496


# Wordpiece tokenizer

-   Byte Pair Encoding 이 빈도 기반이라면 wordpiece tokenizer는 확률 기반으로 글자 쌍을 병합한다.
-   두개 글자 쌍의 빈도수를 각 개별 글자 빈도수의 곱으로 나눈 점수가 가장 높은 순서대로 글자쌍을 묶어 나간다.

$$
score = \cfrac{f(x, y)}{f(x)\cdot f(y)}
$$

함수 f는 빈도를 나타내며 x, y는 병합하려는 하위 단어이다.

-   빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
-   어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w')
-   가장 빈도수가 높은 쌍은 'e','s'로 9번 등장한다. 이때 각 글자는 전체에서 각각 'e'는 17번, 's'는 9번 등장한다. 위 공식에 대입하면 score는 $\frac{9}{17 \times 9} \approx 0.06$ 이다.
-   'i'와 'd' 쌍은 3번만 등장하지만 전체에서 각각 'i' 3번, 'd' 3번 등장한다. 그래서 score는 $\frac{3}{3 \times 3} \approx 0.33$ 이다.
-   나타난 빈도수는 'es' 가 많치만 더 높은 score를 가지는 'id' 쌍을 병합한다.
-   빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', **'id'**, 'e', 's', 't', 3)
-   어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'id'**)
    위의 작업을 반복해 연속된 글자 쌍이 더이상 나타나지 않거나 어휘 사전 max 크기에 도달할 때 까지 학습한다.


## 허깅페이스 Tokenizers 라이브러리 사용

-   Normalization(정규화), 사전토큰화(Pre-tokenization)을 제공
-   정규화
    -   일관된 형식으로 텍스트를 표준화하고, 모호함 제거를 위해 일부 문자 제거 또는 대체 작업을 한다.
-   사전 토큰화
    -   입력문장을 토큰화하기 전에 단어와 같은 작은 단위로 나누는 기능 제공.
-   설치: `pip install tokenizers`


In [35]:
!pip install tokenizers

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 28.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/388.9 kB ? eta -:--:--
   --------------------------------------- 388.9/388.9 kB 25.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/172.0 kB ? eta -:--:--
   ---------------------------------------- 172.0/172.0 kB ? eta 0:00:00


In [37]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece, BPE # Byte Pair Encoding 알고리즘.
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordPieceTrainer, BpeTrainer
import time

path = "data/petitions_corpus.txt"
# 토크나이저 알고리즘 객체를 넣어서 tokenizer 생성.
wp_tokenizer = Tokenizer(WordPiece(unk_token='[UNK]'))  
# Pre tokenizer 설정. (1차 토큰화 작업단위)
wp_tokenizer.pre_tokenizer = Whitespace()
# Trainer 생성 -> 어휘사전 어떻게 만들지 설정. 
trainer = WordPieceTrainer(vocab_size=20000, special_tokens=["[UNK]"])

In [39]:
# 학습 -> 50000줄씩 끊어서 학습.
batch_size = 50_000  # 몇 라인단위로 학습 시킬지.
current_batch = [] # 학습시킬 문자열들을 담을 리스트
text_path = "data/petitions_corpus.txt"
with open(text_path, 'rt', encoding='utf-8') as fr:
    s = time.time()
    # batch_size 줄만큼 current_batch 리스트에 저장 한 뒤 학습.
    for line in fr:
        current_batch.append(line)
        if len(current_batch) == batch_size: # batch size만큼 읽었다면 학습
            # 학습
            wp_tokenizer.train_from_iterator(current_batch, trainer)
            # 리스트 비우기
            current_batch.clear()
    # current_batch의 있는 나머지 데이터 학습
    wp_tokenizer.train_from_iterator(current_batch, trainer)
    e = time.time()

print(f"걸린시간: {e-s}초")
# 저장
saved_path = "saved_models/petitions_wordpiece.json"

걸린시간: 49.07844352722168초


AttributeError: 'SentencePieceProcessor' object has no attribute 'save'

In [41]:
wp_tokenizer.save(saved_path)

In [42]:
# 저장된 model을 loading
saved_tokenizer = Tokenizer.from_file(saved_path)

In [44]:
# 토큰화
output = wp_tokenizer.encode(txt)
print(type(output))

<class 'tokenizers.Encoding'>


In [45]:
# 토큰들
output.tokens

['안녕하세요',
 '.',
 '현재',
 '사대',
 ',',
 '교대',
 '등',
 '교원',
 '##양',
 '##성',
 '##학교',
 '##들의',
 '예비',
 '##교사',
 '##들이',
 '임용',
 '##절',
 '##벽',
 '##에',
 '매우',
 '힘들어',
 '하고',
 '있는',
 '줄',
 '##로',
 '압니다',
 '.',
 '정부',
 '부처',
 '##에서는',
 '영양',
 '##사의',
 '영양',
 "'",
 '교사',
 "'",
 '화',
 ',',
 '폭발',
 '##적인',
 '영양',
 "'",
 '교사',
 "'",
 '채용',
 ',',
 '기간제',
 '교사',
 ',',
 '영',
 '##전',
 '##강',
 ',',
 '스',
 '##강',
 '##의',
 '무기계약직',
 '##화가',
 '그들의',
 '임용',
 '절',
 '##벽',
 '##과는',
 '전혀',
 '무관',
 '##한',
 '일이라고',
 '주장하고',
 '있지만',
 '조금만',
 '생각해보',
 '##면',
 '전혀',
 '설득',
 '##력',
 '없는',
 '말이',
 '##라고',
 '생각합니다',
 '.',
 '학교',
 '수가',
 '같고',
 ',',
 '학생',
 '수가',
 '동일한',
 '##데',
 '영양',
 '##교사',
 '##와',
 '기간제',
 '교사',
 ',',
 '영',
 '##전',
 '##강',
 '스',
 '##강',
 '##이',
 '학교에',
 '늘어나',
 '##게',
 '되면',
 '당연히',
 '정',
 '##규',
 '교원',
 '##의',
 '수는',
 '줄어들',
 '##게',
 '되지',
 '않겠습니까',
 '?',
 '기간제',
 '교사',
 ',',
 '영',
 '##전',
 '##강',
 ',',
 '스',
 '##강',
 '##의',
 '무기계약직',
 '##화',
 ',',
 '정규직',
 '##화',
 '꼭',
 '전면',
 '백',
 '##지',
 '##

In [48]:
output.ids
# 각 토큰의 mapping id로 토큰화

[7628,
 13,
 7228,
 14909,
 11,
 19443,
 1994,
 18442,
 3941,
 3978,
 7347,
 7153,
 9408,
 10015,
 7121,
 14728,
 4040,
 4129,
 3966,
 7889,
 9002,
 7165,
 7125,
 3147,
 4128,
 10699,
 13,
 7163,
 11349,
 7294,
 11202,
 7609,
 11202,
 6,
 8983,
 6,
 3745,
 11,
 16052,
 7173,
 11202,
 6,
 8983,
 6,
 8364,
 11,
 15682,
 8983,
 11,
 2909,
 4028,
 4103,
 11,
 2702,
 4103,
 4023,
 18028,
 8387,
 9058,
 14728,
 3088,
 4129,
 9728,
 7655,
 12002,
 4049,
 17438,
 17920,
 8061,
 11298,
 14836,
 3955,
 7655,
 10620,
 4075,
 7206,
 7517,
 7154,
 7255,
 13,
 7306,
 8176,
 13705,
 11,
 7282,
 8176,
 11671,
 4214,
 11202,
 10015,
 4097,
 15682,
 8983,
 11,
 2909,
 4028,
 4103,
 2702,
 4103,
 3938,
 10149,
 9647,
 3970,
 8201,
 8238,
 3096,
 4382,
 18442,
 4023,
 10242,
 10762,
 3970,
 7802,
 11613,
 30,
 15682,
 8983,
 11,
 2909,
 4028,
 4103,
 11,
 2702,
 4103,
 4023,
 18028,
 4036,
 11,
 8541,
 4036,
 1658,
 10188,
 2392,
 3997,
 4036,
 11341,
 13,
 2392,
 4179,
 4147,
 4154,
 3974,
 8770,
 7308,


In [51]:
len(txt)

1491

In [52]:
from sentencepiece import SentencePieceProcessor
# 토크나이저 생성 및 학습 모델 로딩
tokenizer = SentencePieceProcessor()
tokenizer.load('saved_models/petitions_bpe.model')

True

In [54]:
def bpe_tokenizer(document):
    # 클리닝(텍스트 정제)
    return tokenizer.EncodeAsPieces(document)

In [ ]:
cv = CountVectorizer(
    tokenizer=bpe_tokenizer, # 토큰화 함수를 제공.
    token_pattern=None  # 토큰기준 regexp 패턴. 토크나이저 함수를 지정하므로 default 패턴을 제거.
)
cv.fit()